# Sunlight Plant App - Model Training (Colab)

This notebook is an alternative to running training on your Mac M3 Max.
It runs the species distillation and disease model conversion on a free Colab GPU.

**Steps:**
1. Clone the repo
2. Install dependencies
3. Convert disease model to TFLite
4. Run species distillation (ViT teacher -> MobileNetV2 student)
5. Export to TFLite
6. Download the .tflite model files

In [ ]:
# Step 1: Clone the repo
!git clone https://github.com/jha-vikas/sunleaf.git
%cd sunleaf

In [ ]:
# Step 2: Install dependencies
!pip install torch torchvision transformers huggingface-hub litert-torch pillow numpy tqdm kagglehub

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Step 3: Convert disease model (no GPU needed, ~5 min)
!python training/disease_detection/download_and_convert.py

In [ ]:
# Step 4: Run species distillation (~1-2 hours on Colab T4 GPU)
!python training/species_identification/distill.py --epochs 20 --batch-size 32

In [ ]:
# Step 5: Export species model to TFLite
!python training/species_identification/export_tflite.py

In [ ]:
# Step 6: Run tests to validate
!pip install pytest
!python -m pytest tests/ -v

In [ ]:
# Step 7: Check output files
import os
for f in ['models/plant_disease_v1.tflite', 'models/plant_species.tflite']:
    if os.path.exists(f):
        size_mb = os.path.getsize(f) / (1024 * 1024)
        print(f"{f}: {size_mb:.1f} MB")
    else:
        print(f"{f}: NOT FOUND")

In [ ]:
# Step 8: Download model files to your local machine
from google.colab import files

files.download('models/plant_disease_v1.tflite')
files.download('models/plant_species.tflite')
files.download('models/labels/disease_labels.txt')
files.download('models/labels/species_labels.txt')